In [1]:
import pandas as pd
import datetime

In [2]:
# Using the basic stats as outlined in the dataset description (except urls)
masterMLpublic = pd.read_csv("/kaggle/input/mma-differentials-and-elo/masterMLpublic.csv",
                            usecols = [
                                "date", "result", "fighter", "opponent", "division",
                                "stance", "dob", "method", "total_comp_time", "round", 
                                "time", "referee", "time_format", "reach", "height",
                                "age", "knockdowns", "sub_attempts", "reversals", "control",
                                "takedowns_landed", "takedowns_attempts",
                                "sig_strikes_landed", "sig_strikes_attempts",
                                "total_strikes_landed", "total_strikes_attempts",
                                "head_strikes_landed", "head_strikes_attempts",
                                "body_strikes_landed", "body_strikes_attempts",
                                "leg_strikes_landed", "leg_strikes_attempts",
                                "distance_strikes_landed", "distance_strikes_attempts",
                                "clinch_strikes_landed", "clinch_strikes_attempts",
                                "ground_strikes_landed", "ground_strikes_attempts",
                                "KO_losses", "days_since_last_comp", "lose_streak",
                                "win_streak", "win_loss_ratio", "total_comp_time", "stamina",
                                "num_fights", "trueskill", "elo"
                            ])
# Convert the date columns from object into datetime
masterMLpublic["date"] = pd.to_datetime(masterMLpublic["date"]).dt.date
masterMLpublic["dob"] = pd.to_datetime(masterMLpublic["dob"]).dt.date
masterMLpublic

,date,result,fighter,opponent,stance,dob,division,method,round,time,...,ground_strikes_attempts,num_fights,win_streak,lose_streak,win_loss_ratio,KO_losses,total_comp_time,stamina,elo,trueskill
0,1994-03-11,0,sean daugherty,scott morris,NaN,1975-12-04,Open Weight,SUB,1,0:20,...,0,1.0,0,1,0.000000,0,20.0,0.000000,1147.756618,20.794729
1,1994-03-11,1,scott morris,sean daugherty,Orthodox,NaT,Open Weight,SUB,1,0:20,...,0,1.0,1,0,1.000000,0,20.0,0.000000,1252.243382,29.205271
2,1994-03-11,0,ray wizard,patrick smith,NaN,NaT,Open Weight,SUB,1,0:58,...,0,1.0,0,1,0.000000,0,58.0,0.000000,1147.164966,20.794729
3,1994-03-11,1,patrick smith,ray wizard,Orthodox,1963-08-28,Open Weight,SUB,1,0:58,...,0,1.0,1,0,1.000000,0,58.0,0.000000,1249.442138,29.205271
4,1994-03-11,0,david levicki,johnny rhodes,NaN,NaT,Open Weight,KO/TKO,1,12:13,...,1,1.0,0,1,0.000000,1,733.0,0.000000,1141.838344,20.794729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14125,2023-04-15,1,azamat murzakanov,dustin jacoby,Southpaw,1989-04-12,Light Heavyweight,U-DEC,3,5:00,...,0,3.0,3,0,1.000000,0,2222.0,1.000000,1341.922237,35.760448
14126,2023-04-15,0,billy quarantillo,edson barboza,Orthodox,1988-12-08,Featherweight,KO/TKO,1,2:37,...,0,8.0,0,1,0.625000,1,5352.0,0.600000,1264.744398,31.425332
14127,2023-04-15,1,edson barboza,billy quarantillo,Orthodox,1986-01-21,Featherweight,KO/TKO,1,2:37,...,1,28.0,1,0,0.607143,4,18947.0,0.631579,1350.043973,34.795952
14128,2023-04-15,0,arnold allen,max holloway,Southpaw,1994-01-22,Featherweight,U-DEC,5,5:00,...,0,11.0,0,1,0.909091,0,8823.0,0.888889,1489.695647,39.967508


In [3]:
masterMLpublic.dtypes

# Need to figure out what to do with the objects. Contextual bandit -> linear function (i.e., needs some number)
# 1. Label encode?
# 2. Ignore?

# Objects are:
# date        -> ignore
# fighter     -> ignore
# opponent    -> ignore
# stance      -> label encode
# dob         -> ignore (use age instead)
# division    -> ignore (since we only compare within a division)
# method      -> label encode
# time        -> convert to seconds
# referee     -> ignore
# time_format -> label encode

date                          object
result                         int64
fighter                       object
opponent                      object
stance                        object
dob                           object
division                      object
method                        object
round                          int64
time                          object
referee                       object
time_format                   object
reach                        float64
age                          float64
height                       float64
knockdowns                     int64
days_since_last_comp         float64
sub_attempts                   int64
reversals                      int64
control                        int64
takedowns_landed               int64
takedowns_attempts             int64
sig_strikes_landed             int64
sig_strikes_attempts           int64
total_strikes_landed           int64
total_strikes_attempts         int64
head_strikes_landed            int64
h

In [4]:
masterMLpublic.isna().sum()             # count of na
# masterMLpublic.isna().mean() * 100    # percentage of na

# Need to figure out what to do with null values.

# If {variable} is null -> {action}:
# stance  -> remove
# dob     -> remove
# referee -> remove
# reach   -> impute with division average
# age     -> remove
# height  -> remove

date                            0
result                          0
fighter                         0
opponent                        0
stance                         97
dob                           235
division                        0
method                          0
round                           0
time                            0
referee                        64
time_format                     0
reach                        1282
age                           235
height                         14
knockdowns                      0
days_since_last_comp            0
sub_attempts                    0
reversals                       0
control                         0
takedowns_landed                0
takedowns_attempts              0
sig_strikes_landed              0
sig_strikes_attempts            0
total_strikes_landed            0
total_strikes_attempts          0
head_strikes_landed             0
head_strikes_attempts           0
body_strikes_landed             0
body_strikes_a

In [5]:
# for col in masterMLpublic.columns:
#     print(col)

In [6]:
# masterdataframe = pd.read_csv("/kaggle/input/mma-differentials-and-elo/masterdataframe.csv")
# masterdataframe

In [7]:
# for col in masterdataframe.columns:
#     print(col)

In [8]:
masterMLpublic["division"].value_counts()

division
Lightweight              2470
Welterweight             2410
Middleweight             1898
Featherweight            1340
Heavyweight              1318
Light Heavyweight        1284
Bantamweight             1198
Flyweight                 582
Women's Strawweight       516
Women's Flyweight         380
Women's Bantamweight      362
Open Weight               204
Catch Weight              114
Women's Featherweight      52
Super Heavyweight           2
Name: count, dtype: int64

In [9]:
lightweight = masterMLpublic[masterMLpublic["division"] == "Lightweight"]
lightweight

,date,result,fighter,opponent,stance,dob,division,method,round,time,...,ground_strikes_attempts,num_fights,win_streak,lose_streak,win_loss_ratio,KO_losses,total_comp_time,stamina,elo,trueskill
198,1997-02-07,0,rainy martinez,jerry bohlander,Orthodox,NaT,Lightweight,SUB,1,1:24,...,0,1.0,0,1,0.000000,0,84.0,0.000000,1155.936792,21.246039
199,1997-02-07,1,jerry bohlander,rainy martinez,Orthodox,1974-02-12,Lightweight,SUB,1,1:24,...,0,4.0,2,0,0.750000,1,1858.0,0.000000,1292.341803,29.965180
200,1997-02-07,0,wallid ismail,yoshiki takahashi,Orthodox,1968-02-23,Lightweight,U-DEC,2,3:00,...,0,1.0,0,1,0.000000,0,900.0,0.000000,1147.756618,20.794729
201,1997-02-07,1,yoshiki takahashi,wallid ismail,Southpaw,1969-03-13,Lightweight,U-DEC,2,3:00,...,10,1.0,1,0,1.000000,0,900.0,0.000000,1252.243382,29.205271
206,1997-02-07,0,nick sanzo,jerry bohlander,NaN,NaT,Lightweight,SUB,1,0:39,...,0,1.0,0,1,0.000000,0,39.0,0.000000,1162.881928,21.710767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14055,2023-03-18,1,justin gaethje,rafael fiziev,Orthodox,1988-11-14,Lightweight,M-DEC,3,5:00,...,0,11.0,1,0,0.636364,2,6670.0,0.600000,1491.524082,38.809578
14108,2023-04-15,0,lando vannata,daniel zellhuber,Orthodox,1992-03-14,Lightweight,U-DEC,3,5:00,...,2,13.0,0,2,0.307692,0,9089.0,0.222222,1148.578353,26.265970
14109,2023-04-15,1,daniel zellhuber,lando vannata,Switch,1999-07-07,Lightweight,U-DEC,3,5:00,...,51,2.0,1,0,0.500000,0,1800.0,0.500000,1200.212327,26.476804
14118,2023-04-15,0,clay guida,rafa garcia,Orthodox,1981-12-08,Lightweight,U-DEC,3,5:00,...,0,35.0,0,1,0.514286,2,24276.0,0.521739,1178.957220,31.316986


In [10]:
lightweight["fighter"].nunique()

524

In [11]:
date_threshold = datetime.date(2021, 1, 1)
early_lightweight = lightweight[lightweight["date"] < date_threshold]
late_lightweight = lightweight[lightweight["date"] >= date_threshold]

In [12]:
early_lightweight

,date,result,fighter,opponent,stance,dob,division,method,round,time,...,ground_strikes_attempts,num_fights,win_streak,lose_streak,win_loss_ratio,KO_losses,total_comp_time,stamina,elo,trueskill
198,1997-02-07,0,rainy martinez,jerry bohlander,Orthodox,NaT,Lightweight,SUB,1,1:24,...,0,1.0,0,1,0.000000,0,84.0,0.000,1155.936792,21.246039
199,1997-02-07,1,jerry bohlander,rainy martinez,Orthodox,1974-02-12,Lightweight,SUB,1,1:24,...,0,4.0,2,0,0.750000,1,1858.0,0.000,1292.341803,29.965180
200,1997-02-07,0,wallid ismail,yoshiki takahashi,Orthodox,1968-02-23,Lightweight,U-DEC,2,3:00,...,0,1.0,0,1,0.000000,0,900.0,0.000,1147.756618,20.794729
201,1997-02-07,1,yoshiki takahashi,wallid ismail,Southpaw,1969-03-13,Lightweight,U-DEC,2,3:00,...,10,1.0,1,0,1.000000,0,900.0,0.000,1252.243382,29.205271
206,1997-02-07,0,nick sanzo,jerry bohlander,NaN,NaT,Lightweight,SUB,1,0:39,...,0,1.0,0,1,0.000000,0,39.0,0.000,1162.881928,21.710767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11743,2020-12-05,1,gabriel benitez,justin jaynes,Southpaw,1988-06-15,Lightweight,KO/TKO,1,4:06,...,5,10.0,1,0,0.600000,2,5342.0,0.600,1273.013267,29.423404
11754,2020-12-12,0,renato moicano,rafael fiziev,Orthodox,1989-05-21,Lightweight,KO/TKO,1,4:05,...,0,10.0,0,1,0.600000,3,4935.0,0.750,1240.856956,34.152111
11755,2020-12-12,1,rafael fiziev,renato moicano,Switch,1993-03-05,Lightweight,KO/TKO,1,4:05,...,1,4.0,3,0,0.750000,1,2131.0,1.000,1300.963413,33.694983
11764,2020-12-12,0,tony ferguson,charles oliveira,Orthodox,1984-02-12,Lightweight,U-DEC,3,5:00,...,3,18.0,0,2,0.833333,1,11985.0,0.625,1506.243446,38.479987


In [13]:
late_lightweight

,date,result,fighter,opponent,stance,dob,division,method,round,time,...,ground_strikes_attempts,num_fights,win_streak,lose_streak,win_loss_ratio,KO_losses,total_comp_time,stamina,elo,trueskill
11816,2021-01-20,0,mason jones,mike davis,Orthodox,1995-04-26,Lightweight,U-DEC,3,5:00,...,7,1.0,0,1,0.000000,0,900.0,-0.050000,1165.782229,20.474403
11817,2021-01-20,1,mike davis,mason jones,Orthodox,1992-10-07,Lightweight,U-DEC,3,5:00,...,4,3.0,2,0,0.666667,0,2340.0,1.000000,1246.438167,27.871592
11850,2021-01-23,0,matt frevola,arman tsarukyan,Orthodox,1990-06-11,Lightweight,U-DEC,3,5:00,...,0,5.0,0,1,0.400000,1,3660.0,0.500000,1186.406302,29.233458
11851,2021-01-23,1,arman tsarukyan,matt frevola,Orthodox,1996-10-11,Lightweight,U-DEC,3,5:00,...,9,4.0,3,0,0.750000,0,3600.0,0.750000,1343.102444,35.430054
11858,2021-01-23,0,dan hooker,michael chandler,Switch,1990-02-13,Lightweight,KO/TKO,1,2:30,...,0,16.0,0,2,0.625000,2,9945.0,0.375000,1338.823540,33.640663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14055,2023-03-18,1,justin gaethje,rafael fiziev,Orthodox,1988-11-14,Lightweight,M-DEC,3,5:00,...,0,11.0,1,0,0.636364,2,6670.0,0.600000,1491.524082,38.809578
14108,2023-04-15,0,lando vannata,daniel zellhuber,Orthodox,1992-03-14,Lightweight,U-DEC,3,5:00,...,2,13.0,0,2,0.307692,0,9089.0,0.222222,1148.578353,26.265970
14109,2023-04-15,1,daniel zellhuber,lando vannata,Switch,1999-07-07,Lightweight,U-DEC,3,5:00,...,51,2.0,1,0,0.500000,0,1800.0,0.500000,1200.212327,26.476804
14118,2023-04-15,0,clay guida,rafa garcia,Orthodox,1981-12-08,Lightweight,U-DEC,3,5:00,...,0,35.0,0,1,0.514286,2,24276.0,0.521739,1178.957220,31.316986
